In [54]:
#!pip install ipywidgets

In [55]:
import requests
import json
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import pprint
import re
import nbformat
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from datetime import datetime
from ipywidgets import interact, widgets
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [56]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
with open("../DataProcessing/TestData/postGame.json", "r", encoding="utf-8") as file:
    postData = json.load(file)
    
with open("../DataProcessing/TestData/runeSheet.json", "r", encoding="utf-8") as file:
    runeSheet = json.load(file)
    
participantsData = data['metadata']['participants']
framesData = data['info']['frames']
participantFrames = [frame['participantFrames'] for frame in framesData]

championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

mostMentionedNamesDict = {}
participantData = []
participantChampionMapping = {}


In [57]:
targetPuuid = '7hSyI8OKEFPpadVyhqKmTOJlT6umda2FMt1sAeIcZ1WcxdATw7Tvu4ixaR8llqv9u2qPutnAsL9Shg'

In [58]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)
    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]
    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

In [59]:
def addParticipantData(participantId, puuid, championName):  # add championName
    participantData.append([participantId, puuid, championName])  # add championName

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            
            championName = mostMentionedNamesDict[participantId]
            addParticipantData(participantId, puuid, championName)
findParticipantData(data)

def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName = entry
        if puuid == targetPuuid:
            return participantId, championName
    return None, None

targetParticipantId, targetChampionName = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}")
else:
    print("Participant not found.")

Participant ID: 2, Champion: MasterYi


In [60]:
def excludeColumns(data, columnsToExclude=None, displayNoZero=False):
    # Exclude specified columns
    excludedData = {}

    for key, value in data.items():
        if key in columnsToExclude:
            continue  # Skip excluded columns
        elif isinstance(value, dict):
            # Recursively exclude columns within nested dictionaries
            excludedData[key] = excludeColumns(value, columnsToExclude, displayNoZero)
        else:
            excludedData[key] = value
    return excludedData

def displayNonZeroParams(participantFrame, displayNoZero=False, indent=''):
    # Iterate through the participant frame's items
    for key, value in participantFrame.items():
        if isinstance(value, dict):
            # If the value is a nested dictionary, iterate through its items
            print(f"{indent}{key} for {targetChampionName}: ")
            displayNonZeroParams(value, displayNoZero, indent + '  ')
        elif isinstance(value, (int, float)) and (not displayNoZero or value != 0):
            # Display the parameter if it's a numeric value and not zero
            print(f"{indent}  {key}: {value}")

def getParticipantDataAtMinute(data, participantId, targetMinute, columnsToExclude=None, displayNoZero=False):
    timestampData = []

    # Check if there are frames in the data
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000)  # Convert milliseconds to minutes and round to the nearest minute

            if currentMinute == targetMinute:
                participantFrames = frame.get('participantFrames', {})
                if participantId in participantFrames:
                    participantData = participantFrames[participantId]
                    timestampData.append([timestamp, participantData])

                    # Call the excludeColumns method
                    excludedData = excludeColumns(participantData, columnsToExclude, displayNoZero)

                    # Display either the original data or the excluded data based on your requirement
                    if displayNoZero:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(excludedData, displayNoZero)
                        print("\n---\n")
                    else:
                        print(f"Timestamp at minute {targetMinute}: {timestamp}")
                        displayNonZeroParams(participantData, displayNoZero)
                        print("\n---\n")
    return timestampData

desiredParticipantId = (f"{targetParticipantId}")
desiredMinute = 2 
columnsToExclude = ['magicDamageDone', 'magicDamageDoneToChampions', 'magicDamageTaken', 
                    'physicalDamageDone', 'physicalDamageDoneToChampions', 'physicalDamageTaken', 
                    'trueDamageDone', 'trueDamageDoneToChampions', 'trueDamageTaken', 'timeEnemySpentControlled']

# Call the function with displayNoZero set to True
#result = getParticipantDataAtMinute(data, desiredParticipantId, desiredMinute, columnsToExclude=columnsToExclude, displayNoZero=True)


In [61]:
def getKillDataForParticipant(data, participantId, participantData):
    killData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if killerId == participantId:
                        timestamp = round(event['timestamp'] / 60000, 2)
                        victimDamageReceived = event.get('victimDamageReceived', [])
                        killPosition = event.get('position', {})  # Added line

                        # Calculate total damage done by the killer to the victim
                        totalDamageDone = sum(
                            damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                            for damageInfo in victimDamageReceived
                            if damageInfo['participantId'] == participantId
                        )

                        # Find names from participantData
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                           f"Unknown Champion with ID {killerId}")
                        victimName = next((participant[2] for participant in participantData if participant[0] == victimId),
                                           f"Unknown Champion with ID {victimId}")

                        # Find names of assisting champions and their damage
                        assistingChampions = []
                        damageByAssists = []

                        for assistId in assistIds:
                            assistName = next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}")

                            assistDamage = sum(
                                damageInfo['basic'] + damageInfo['magicDamage'] + damageInfo['physicalDamage'] + damageInfo['trueDamage']
                                for damageInfo in victimDamageReceived
                                if damageInfo['participantId'] == assistId
                            )

                            assistingChampions.append(assistName)
                            damageByAssists.append(assistDamage)

                        killData.append({
                            'timestamp': timestamp,
                            'killerId': killerId,
                            'killerName': killerName,
                            'victimId': victimId,
                            'victimName': victimName,
                            'totalDamageDone': totalDamageDone,
                            'assistingChampions': assistingChampions,
                            'damageByAssists': damageByAssists,
                            'position': killPosition  # Added line
                        })
    return killData

killDataForParticipant = getKillDataForParticipant(data, targetParticipantId, participantData)

def getAssistsForParticipant(data, targetParticipantId, participantData):
    assistData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    assistIds = [int(assistId) for assistId in event.get('assistingParticipantIds', [])]

                    if targetParticipantId in assistIds:
                        timestamp = round(event['timestamp'] /60000, 2)
                        
                        # Get the name and ID of the killer
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        killerParticipantId = killerId

                        assistData.append({
                            'timestamp': timestamp,
                            'killerId': killerParticipantId,
                            'killerName': killerName,
                            'assistId': targetParticipantId,
                            'assistName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'position': event.get('position', {'x': 0, 'y': 0})  # Use default coordinates if not present
                        })
    return assistData

assistDataForParticipant = getAssistsForParticipant(data, targetParticipantId, participantData)

def getDeathsForParticipant(data, targetParticipantId, participantData):
    deathData = []

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    victimId = int(event.get('victimId'))
                    killerId = int(event.get('killerId'))

                    if victimId == targetParticipantId:
                        timestamp = round(event['timestamp'] / 60000, 2)

                        # Get the name and ID of the killer
                        killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                          f"Unknown Champion with ID {killerId}")

                        killerParticipantId = killerId

                        deathData.append({
                            'timestamp': timestamp,
                            'victimId': targetParticipantId,
                            'victimName': next((participant[2] for participant in participantData if participant[0] == targetParticipantId),
                                               f"Unknown Champion with ID {targetParticipantId}"),
                            'killerId': killerParticipantId,
                            'killerName': killerName,
                            'position': event.get('position', {'x': 0, 'y': 0})  # Use default coordinates if not present
                        })
    return deathData

#deathLocation = f"X: {death['position'].get('x', 0)}, Y: {death['position'].get('y', 0)}"

deathDataForParticipant = getDeathsForParticipant(data, targetParticipantId, participantData)

In [62]:
for assist in assistDataForParticipant:
    assistLocation = f"X: {assist['position'].get('x', 0)}, Y: {assist['position'].get('y', 0)}"

    print(f"Timestamp: {assist['timestamp']}\n"
          f"Killer: {assist['killerName']} (ID: {assist['killerId']})\n"
          f"Assist: {assist['assistName']} (ID: {assist['assistId']})\n"
          f"Assist Location: ({assistLocation})\n")

Timestamp: 4.53
Killer: Jax (ID: 1)
Assist: MasterYi (ID: 2)
Assist Location: (X: 3464, Y: 13074)

Timestamp: 8.98
Killer: Leona (ID: 5)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12205, Y: 3217)

Timestamp: 9.0
Killer: Leona (ID: 5)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12592, Y: 2977)

Timestamp: 13.62
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12998, Y: 4356)

Timestamp: 14.12
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 11970, Y: 4447)

Timestamp: 15.58
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 12644, Y: 2564)

Timestamp: 18.03
Killer: Jax (ID: 1)
Assist: MasterYi (ID: 2)
Assist Location: (X: 4765, Y: 9700)

Timestamp: 18.1
Killer: Veigar (ID: 3)
Assist: MasterYi (ID: 2)
Assist Location: (X: 4053, Y: 9372)

Timestamp: 21.52
Killer: Varus (ID: 4)
Assist: MasterYi (ID: 2)
Assist Location: (X: 9537, Y: 5506)

Timestamp: 21.55
Killer: Veigar (ID: 3)
Assist: MasterYi (ID: 2)
Assist Location: (X: 9610, Y

In [63]:
for kill in killDataForParticipant:
    
    killLocation = f"X: {kill['position'].get('x', 0)}, Y: {kill['position'].get('y', 0)}"

    assistingChampionsLine = f"Assisting Champions: {', '.join(kill['assistingChampions'])}.\n" if kill['assistingChampions'] else ""
    damageByAssistsLine = f"Damage Done by Assists: {', '.join([f'{damage} by {assist}' for damage, assist in zip(kill['damageByAssists'], kill['assistingChampions'])])}.\n" if kill['assistingChampions'] else ""

    print(f"Timestamp: {kill['timestamp']}\n"
          f"Killer: {kill['killerName']} (ID: {kill['killerId']})\n"
          f"Victim: {kill['victimName']} (ID: {kill['victimId']})\n"
          f"Total Damage Done: {kill['totalDamageDone']}\n"
          f"{assistingChampionsLine}"
          f"{damageByAssistsLine}"
          f"Kill Location: ({killLocation})\n")

Timestamp: 6.41
Killer: MasterYi (ID: 2)
Victim: Khazix (ID: 7)
Total Damage Done: 998
Kill Location: (X: 9400, Y: 6528)

Timestamp: 17.94
Killer: MasterYi (ID: 2)
Victim: Shen (ID: 6)
Total Damage Done: 419
Assisting Champions: Jax, Veigar.
Damage Done by Assists: 702 by Jax, 1168 by Veigar.
Kill Location: (X: 4455, Y: 9895)

Timestamp: 19.31
Killer: MasterYi (ID: 2)
Victim: Sylas (ID: 8)
Total Damage Done: 1397
Assisting Champions: Jax, Veigar.
Damage Done by Assists: 723 by Jax, 0 by Veigar.
Kill Location: (X: 4071, Y: 8585)

Timestamp: 20.32
Killer: MasterYi (ID: 2)
Victim: Khazix (ID: 7)
Total Damage Done: 1934
Kill Location: (X: 9128, Y: 1327)

Timestamp: 21.93
Killer: MasterYi (ID: 2)
Victim: Nautilus (ID: 10)
Total Damage Done: 1272
Assisting Champions: Veigar, Varus, Leona.
Damage Done by Assists: 295 by Veigar, 540 by Varus, 69 by Leona.
Kill Location: (X: 8680, Y: 6775)

Timestamp: 22.01
Killer: MasterYi (ID: 2)
Victim: Sylas (ID: 8)
Total Damage Done: 844
Assisting Champion

In [64]:
def getTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            takedowns = participant.get('challenges', {}).get('takedowns', None)
            if takedowns is not None:
                print(f"  Takedowns by {targetChampionName}: {takedowns}")
            else:
                print(f"No Data for Takedowns by {targetChampionName}")
    return None

def getKda(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            kda = participant.get('challenges', {}).get('kda', None)
            if kda is not None:
                print(f"  KDA by {targetChampionName}: {kda}\n")
            else:
                print(f"No KDA found for {targetChampionName}\n")
    return None

def getSoloKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            soloKills = participant.get('challenges', {}).get('soloKills', None)
            if soloKills is not None:
                print(f"  Solo Kills for {targetChampionName}: {soloKills}")
            else:
                print(f"No kill participation found for {targetChampionName}")
    return None
    
def getKillParticipation(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            killParticipation = participant.get('challenges', {}).get('killParticipation', None)
            if killParticipation is not None:
                print(f"  Kill Participation for {targetChampionName}: {killParticipation}")
            else:
                print(f"No kill participation found for {targetChampionName}")
            return killParticipation
    return None    

def getLaneRole(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            laneRole = participant.get('individualPosition', None)
            if laneRole is not None:
                print(f"Role of {targetChampionName}: {laneRole}")
            return laneRole
    return None  
    
def getJungleStats(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid and participant.get('individualPosition') == 'JUNGLE':
            totalAllyJungleMinionsKilled = participant.get('totalAllyJungleMinionsKilled', None)
            totalEnemyJungleMinionsKilled = participant.get('totalEnemyJungleMinionsKilled', None)
            if totalAllyJungleMinionsKilled is not None and totalEnemyJungleMinionsKilled is not None:
                print(f"  Total Ally Jungle Minions Killed: {totalAllyJungleMinionsKilled}")
                print(f"  Total Enemy Jungle Minions Killed: {totalEnemyJungleMinionsKilled}\n")
            return totalAllyJungleMinionsKilled, totalEnemyJungleMinionsKilled
    return None, None

def getTeamSide(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamSide = participant.get('teamId', None)
            if teamSide is not None:
                if teamSide == 100:
                    print(f"{targetChampionName} was Blue Side")
                else:
                    print(f"{targetChampionName} was Red Side")
            return teamSide
    return None

def getGameWon(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            gameWon = participant.get('win', None)
            if gameWon is not None:
                if gameWon == True:
                    print(f"{targetChampionName}s Team Won the Game")
                else:
                    print(f"{targetChampionName}s Team Lost the Game")        
            return
    return None

def getTotalDamageDealt(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageDealt = participant.get('totalDamageDealt', None)
            if totalDamageDealt is not None:
                print(f"  Total Damage Dealt by {targetChampionName}: {totalDamageDealt}")
    return None

def getTotalDamageDealtToChampions(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageDealtToChampions = participant.get('totalDamageDealtToChampions', None)
            if totalDamageDealtToChampions is not None:
                print(f"  Total Damage Dealt To Champions by {targetChampionName}: {totalDamageDealtToChampions}")
    return None

def getTotalDamageTaken(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            totalDamageTaken = participant.get('totalDamageTaken', None)
            if totalDamageTaken is not None:
                print(f"  Total Damage Taken for {targetChampionName}: {totalDamageTaken}\n")
    return None

def getDeathsByEnemyChamps(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            deathsByEnemyChamps =  participant.get('challenges', {}).get('deathsByEnemyChamps', None)
            if deathsByEnemyChamps is not None:
                print(f"  Deaths by Enemy Champions on {targetChampionName}: {deathsByEnemyChamps}")
    return None

def getTeamBaronKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamBaronKills = participant.get('challenges', {}).get('teamBaronKills', None)
            if teamBaronKills is not None and teamBaronKills != 0:
                print(f"  Team Baron Kills by {targetChampionName}: {teamBaronKills}")
    return None

def getTeamRiftHeraldKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamRiftHeraldKills = participant.get('challenges', {}).get('teamRiftHeraldKills', None)
            if teamRiftHeraldKills is not None and teamRiftHeraldKills != 0:
                print(f"  Team Rift Herald Kills by {targetChampionName}: {teamRiftHeraldKills}")
    return None

def getTeamElderDragonKills(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            teamElderDragonKills = participant.get('challenges', {}).get('teamElderDragonKills', None)
            if teamElderDragonKills is not None and teamElderDragonKills != 0:
                print(f"  Team Elder Dragon Kills by {targetChampionName}: {teamElderDragonKills}")
    return None

def getDragonTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            dragonTakedowns = participant.get('challenges', {}).get('dragonTakedowns', None)
            if dragonTakedowns is not None and dragonTakedowns != 0:
                print(f"  Dragon Takedowns by {targetChampionName}: {dragonTakedowns}")
    return None

def getVisionScoreAdvantageLaneOpponent(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            visionScoreAdvantage = participant.get('challenges', {}).get('visionScoreAdvantageLaneOpponent')
            if visionScoreAdvantage is not None:
                print(f"  Vision Score Advantage for {targetChampionName}: {visionScoreAdvantage}")
    return None

def getTurretTakedowns(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            turretTakedowns = participant.get('challenges', {}).get('turretTakedowns', None)
            if turretTakedowns is not None and turretTakedowns != 0:
                print(f"  Turret Takedowns by {targetChampionName}: {turretTakedowns}\n")  
    return None

def getVisionScore(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            visionScore = participant.get('visionScore', None)
            if visionScore is not None:
                print(f"  Vison Score for {targetChampionName}: {visionScore}")
    return None

def getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            controlWardTimeCoverageInRiverOrEnemyHalf = participant.get('challenges', {}).get('controlWardTimeCoverageInRiverOrEnemyHalf', None)
            if controlWardTimeCoverageInRiverOrEnemyHalf is not None and controlWardTimeCoverageInRiverOrEnemyHalf != 0:
                print(f"  Control Ward Time Coverage In River or Enemy Half for {targetChampionName}: {controlWardTimeCoverageInRiverOrEnemyHalf}")
    return None

def getControlWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            controlWardsPlaced = participant.get('challenges', {}).get('controlWardsPlaced', None)
            if controlWardsPlaced is not None and controlWardsPlaced != 0:
                print(f"  Control Wards placed by {targetChampionName}: {controlWardsPlaced}")
    return None

def getStealthWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            stealthWardsPlaced = participant.get('challenges', {}).get('stealthWardsPlaced', None)
            if stealthWardsPlaced is not None and stealthWardsPlaced != 0:
                print(f"  Stealth Wards Placed by {targetChampionName}: {stealthWardsPlaced}")
    return None

def getWardsKilled(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            wardsKilled = participant.get('wardsKilled', None)
            if wardsKilled is not None and wardsKilled != 0:
                print(f"  Wards Killed by {targetChampionName}: {wardsKilled}\n")
    return None

def getWardsPlaced(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            wardsPlaced = participant.get('wardsPlaced', None)
            if wardsPlaced is not None and wardsPlaced != 0:
                print(f"  Wards Placed by {targetChampionName}: {wardsPlaced}")
    return None

def getLaneMinionsFirst10Minutes(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            laneMinionsFirst10Minutes = participant.get('challenges', {}).get('laneMinionsFirst10Minutes', None)
            if laneMinionsFirst10Minutes is not None and getLaneRole != "JUNGLE":
                print(f"  Lane Minions First 10 Minutes by {targetChampionName}: {laneMinionsFirst10Minutes}\n") 
    return None

def getGoldPerMinute(postData, targetPuuid):
    for participant in postData['info']['participants']:
        if participant.get('puuid') == targetPuuid:
            goldPerMinute = participant.get('challenges', {}).get('goldPerMinute', None)
            if goldPerMinute is not None:
                print(f"  Gold Per Minute for {targetChampionName}: {goldPerMinute}")
    return None

getLaneRole(postData, targetPuuid)
getJungleStats(postData, targetPuuid)

getSoloKills(postData, targetPuuid)
getKillParticipation(postData, targetPuuid)
getTakedowns(postData, targetPuuid)
getDeathsByEnemyChamps(postData, targetPuuid)
getKda(postData, targetPuuid)

getTotalDamageDealt(postData, targetPuuid)
getTotalDamageDealtToChampions(postData, targetPuuid)
getTotalDamageTaken(postData, targetPuuid)

getTeamBaronKills(postData, targetPuuid)
getTeamRiftHeraldKills(postData, targetPuuid)
getDragonTakedowns(postData, targetPuuid)
getTeamElderDragonKills(postData, targetPuuid)
getTurretTakedowns(postData, targetPuuid)

getVisionScore(postData, targetPuuid)
getVisionScoreAdvantageLaneOpponent(postData, targetPuuid)

getWardsPlaced(postData, targetPuuid)
getStealthWardsPlaced(postData, targetPuuid)
getControlWardsPlaced(postData, targetPuuid)
getControlWardTimeCoverageInRiverOrEnemyHalf(postData, targetPuuid)
getWardsKilled(postData, targetPuuid)

getGoldPerMinute(postData,targetPuuid)
getLaneMinionsFirst10Minutes(postData, targetPuuid)

getTeamSide(postData, targetPuuid)
getGameWon(postData, targetPuuid)

Role of MasterYi: JUNGLE
  Total Ally Jungle Minions Killed: 92
  Total Enemy Jungle Minions Killed: 16

  Solo Kills for MasterYi: 5
  Kill Participation for MasterYi: 0.68
  Takedowns by MasterYi: 34
  Deaths by Enemy Champions on MasterYi: 12
  KDA by MasterYi: 2.8333333333333335

  Total Damage Dealt by MasterYi: 292732
  Total Damage Dealt To Champions by MasterYi: 51385
  Total Damage Taken for MasterYi: 49204

  Team Baron Kills by MasterYi: 1
  Dragon Takedowns by MasterYi: 2
  Team Elder Dragon Kills by MasterYi: 1
  Turret Takedowns by MasterYi: 1

  Vison Score for MasterYi: 38
  Vision Score Advantage for MasterYi: 0.44010066986083984
  Wards Placed by MasterYi: 9
  Stealth Wards Placed by MasterYi: 7
  Control Wards placed by MasterYi: 2
  Control Ward Time Coverage In River or Enemy Half for MasterYi: 0.44759596864850204
  Wards Killed by MasterYi: 2

  Gold Per Minute for MasterYi: 498.52359737296433
  Lane Minions First 10 Minutes by MasterYi: 2

MasterYi was Blue Side


In [66]:
def getEventsInMinute(data, targetMinute):
    eventsInMinute = []
    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            timestamp = frame['timestamp']
            currentMinute = round(timestamp / 60000, 2)  # Round to two decimal places
            if currentMinute == targetMinute:
                eventsInMinute.extend(frame.get('events', []))
    return eventsInMinute

In [67]:
def getTowerDestroyedData(data, participantData):
    towerData = []  # Rename the variable for consistency

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', None)
                timestamp = event['timestamp']
                currentMinute = round(timestamp / 60000, 2)
                position = event['position']
                laneType = event['laneType']
                killerId = event['killerId']
                assistingParticipantIds = event.get('assistingParticipantIds', [])

                # Retrieve champion name for the killer
                killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                  f"Minions")

                # Retrieve champion names for assisting participants only if there are assisting participants
                assistingChampionNames = [next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}") for assistId in assistingParticipantIds]

                towerType = towerType if towerType in ['OUTER_TURRET', 'INNER_TURRET'] else None
                
                towerData.append({
                    'teamId': teamId,
                    'towerType': towerType,  # Keep towerType as it is
                    'timestamp': currentMinute,
                    'position': position,
                    'laneType': laneType,
                    'killerId': killerId,
                    'killerName': killerName,
                    'assistingParticipantIds': assistingParticipantIds,
                    'assistingChampionNames': assistingChampionNames
                })

    return towerData

# Example usage:
towerData = getTowerDestroyedData(data, participantData)

sortedTowerData = sorted(towerData, key=lambda x: (x['teamId'], x['timestamp']))

# Display the extracted data
for tower in sortedTowerData:
    currentMinute = tower['timestamp']
    towerType = tower['towerType']
    
    # Check if towerType is 'OUTER_TURRET' or 'INNER_TURRET' before printing
    if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
        print(f"Team: {tower['teamId']}, Tower Type: {towerType}, Timestamp: {currentMinute}, Lane: {tower['laneType']}")
        print(f"Killer: {tower['killerName']}")
    
        # Print assisting champions only if there are assisting participants
        if tower['assistingParticipantIds']:
            print(f"Assisting Champions: {tower['assistingChampionNames']}")
    
        print('\n')



Team: 100, Tower Type: OUTER_TURRET, Timestamp: 20.04, Lane: BOT_LANE
Killer: Ezreal
Assisting Champions: ['Nautilus']


Team: 100, Tower Type: OUTER_TURRET, Timestamp: 20.91, Lane: MID_LANE
Killer: Sylas


Team: 100, Tower Type: OUTER_TURRET, Timestamp: 29.4, Lane: TOP_LANE
Killer: Minions


Team: 100, Tower Type: INNER_TURRET, Timestamp: 29.9, Lane: TOP_LANE
Killer: Minions
Assisting Champions: ['Shen']


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 17.34, Lane: MID_LANE
Killer: Veigar


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 20.36, Lane: TOP_LANE
Killer: Jax


Team: 200, Tower Type: INNER_TURRET, Timestamp: 21.02, Lane: TOP_LANE
Killer: Jax


Team: 200, Tower Type: OUTER_TURRET, Timestamp: 24.95, Lane: BOT_LANE
Killer: MasterYi


Team: 200, Tower Type: INNER_TURRET, Timestamp: 28.63, Lane: MID_LANE
Killer: Varus


Team: 200, Tower Type: INNER_TURRET, Timestamp: 37.34, Lane: BOT_LANE
Killer: Minions




In [68]:
gameStartTimestamp = postData['info']['gameStartTimestamp']
gameEndTimestamp = postData['info']['gameEndTimestamp']
gameDurationMs = gameEndTimestamp - gameStartTimestamp
gameDurationMinutes = gameDurationMs / 60000

In [69]:
def plotEventsAtMinute(backgroundImagePath, killData, assistData, deathData, towerData, targetMinute, title):
    plt.figure()
    plt.title(f'{title} at Minute {targetMinute}')

    # Load background image
    backgroundImage = mpimg.imread(backgroundImagePath)
    plt.xlim(0, 15000)
    plt.ylim(0, 15000)
    plt.imshow(backgroundImage, extent=[0, 15000, 0, 15000], alpha=0.8)

    # Combine kill, assist, death, and tower data
    allEvents = killData + assistData + deathData + towerData

    # Filter events for the specified minute
    minuteEvents = [event for event in allEvents if event['timestamp'] <= targetMinute]
    
    # Print kills, assists, and deaths for the target participant in the specified minute
    targetKills = [event for event in minuteEvents if 'killerId' in event and event['killerId'] == targetParticipantId]
    targetAssists = [event for event in minuteEvents if 'assistId' in event and event['assistId'] == targetParticipantId]
    targetDeaths = [event for event in minuteEvents if 'victimId' in event and event['victimId'] == targetParticipantId]
    towerDestroyed = [event for event in minuteEvents if event.get('towerType') in ['OUTER_TURRET', 'INNER_TURRET']]

    print(f'\nKills for {targetChampionName} in minute {targetMinute}:')
    for kill in targetKills:
        print(f"Timestamp: {kill['timestamp']}, Victim: {kill.get('victimName', 'Unknown')}")

    print(f'\nAssists for {targetChampionName} in minute {targetMinute}:')
    for assist in targetAssists:
        print(f"Timestamp: {assist['timestamp']}, Killer: {assist.get('killerName', 'Unknown')}")

    print(f'\nDeaths for {targetChampionName} in minute {targetMinute}:')
    for death in targetDeaths:
        print(f"Timestamp: {death['timestamp']}, Killer: {death.get('killerName', 'Unknown')}")

    print(f'\nTowers Destroyed in minute {targetMinute}:')
    for tower in towerDestroyed:
        x = tower['position']['x']
        y = tower['position']['y']
        print(f"Timestamp: {tower['timestamp']}, Killer: {tower['killerName']}, Tower Position: ({x}, {y})")  # Print tower position for debugging
        plt.scatter(x, y, color='yellow', marker='d')

    # Scatter plot for kills, assists, and deaths
    for event in minuteEvents:
        x = event['position']['x']
        y = event['position']['y']

        if 'killerId' in event and event['killerId'] == targetParticipantId:
            plt.scatter(x, y, color='blue', marker='o')
        elif 'assistId' in event and event['assistId'] == targetParticipantId:
            plt.scatter(x, y, color='green', marker='*')
        elif 'victimId' in event and event['victimId'] == targetParticipantId:
            plt.scatter(x, y, color='red', marker='X')

    plt.show()


# Example usage:
# Assuming killDataForParticipant, assistDataForParticipant, deathDataForParticipant,
# and towerDataForParticipant contain the relevant events for kills, assists, deaths,
# and towers destroyed for the target participant
# You can specify the minute you want to see

interact(
    plotEventsAtMinute,
    backgroundImagePath=widgets.fixed('../DataProcessing/TestData/lolMap.PNG'),
    killData=widgets.fixed(killDataForParticipant),
    assistData=widgets.fixed(assistDataForParticipant),
    deathData=widgets.fixed(deathDataForParticipant),
    towerData=widgets.fixed(towerData),
    targetMinute=widgets.FloatSlider(value=21, min=0, max=gameDurationMinutes, step=1, description='Minute:', readout_format='.1f'),
    title=widgets.fixed('KDA Locations')
)


interactive(children=(FloatSlider(value=21.0, description='Minute:', max=39.13223333333333, readout_format='.1…

<function __main__.plotEventsAtMinute(backgroundImagePath, killData, assistData, deathData, towerData, targetMinute, title)>

In [70]:
def extractBuildingKillData(data, participantData):
    buildingKillData = []
    turret_combinations = {}  # Store turret combinations and their corresponding cases

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', 'Unknown Tower Type')
                laneType = event['laneType']

                # Combine relevant information into a key
                turret_key = (teamId, towerType, laneType)

                if turret_key not in turret_combinations:
                    # Assign a new case if the combination is encountered for the first time
                    turret_combinations[turret_key] = len(turret_combinations) + 1

                case = turret_combinations[turret_key]

                timestamp = event['timestamp']
                currentMinute = round(timestamp / 60000, 2)
                position = event['position']
                killerId = event['killerId']
                assistingParticipantIds = event.get('assistingParticipantIds', [])

                # Retrieve champion name for the killer
                killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                  f"Unknown Champion with ID {killerId}")

                # Retrieve champion names for assisting participants only if there are assisting participants
                assistingChampionNames = [next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}") for assistId in assistingParticipantIds]

                buildingKillData.append({
                    'case': case,
                    'teamId': teamId,
                    'towerType': towerType,
                    'laneType': laneType,
                    'timestamp': currentMinute,
                    'position': position,
                    'killerId': killerId,
                    'killerName': killerName,
                    'assistingParticipantIds': assistingParticipantIds,
                    'assistingChampionNames': assistingChampionNames
                })

    return buildingKillData

# Example usage:
buildingKillData = extractBuildingKillData(data, participantData)

# Display the extracted data
for kill in buildingKillData:
    print(f"Case: {kill['case']}")
    print(f"Team: {kill['teamId']}, Tower Type: {kill['towerType']}, Lane: {kill['laneType']}, Timestamp: {kill['timestamp']}")
    print(f"Killer: {kill['killerName']}")
    
    # Print assisting champions only if there are assisting participants
    if kill['assistingParticipantIds']:
        print(f"Assisting Champions: {kill['assistingChampionNames']}")
    
    print('\n')


Case: 1
Team: 200, Tower Type: OUTER_TURRET, Lane: MID_LANE, Timestamp: 17.34
Killer: Veigar


Case: 2
Team: 100, Tower Type: OUTER_TURRET, Lane: BOT_LANE, Timestamp: 20.04
Killer: Ezreal
Assisting Champions: ['Nautilus']


Case: 3
Team: 200, Tower Type: OUTER_TURRET, Lane: TOP_LANE, Timestamp: 20.36
Killer: Jax


Case: 4
Team: 100, Tower Type: OUTER_TURRET, Lane: MID_LANE, Timestamp: 20.91
Killer: Sylas


Case: 5
Team: 200, Tower Type: INNER_TURRET, Lane: TOP_LANE, Timestamp: 21.02
Killer: Jax


Case: 6
Team: 200, Tower Type: OUTER_TURRET, Lane: BOT_LANE, Timestamp: 24.95
Killer: MasterYi


Case: 7
Team: 200, Tower Type: INNER_TURRET, Lane: MID_LANE, Timestamp: 28.63
Killer: Varus


Case: 8
Team: 100, Tower Type: OUTER_TURRET, Lane: TOP_LANE, Timestamp: 29.4
Killer: Unknown Champion with ID 0


Case: 9
Team: 100, Tower Type: INNER_TURRET, Lane: TOP_LANE, Timestamp: 29.9
Killer: Unknown Champion with ID 0
Assisting Champions: ['Shen']


Case: 10
Team: 200, Tower Type: BASE_TURRET, Lane

In [71]:
def organizeBuildingKillData(data, participantData):
    blueTeamTurrets = []
    redTeamTurrets = []

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', 'Unknown Tower Type')  # Use a default value if 'towerType' is not present
                timestamp = event['timestamp']
                currentMinute = round(timestamp / 60000, 2)
                position = event['position']
                laneType = event['laneType']
                killerId = event['killerId']
                assistingParticipantIds = event.get('assistingParticipantIds', [])

                # Retrieve champion name for the killer
                killerName = next((participant[2] for participant in participantData if participant[0] == killerId),
                                  f"Unknown Champion with ID {killerId}")

                # Retrieve champion names for assisting participants only if there are assisting participants
                assistingChampionNames = [next((participant[2] for participant in participantData if participant[0] == assistId),
                                               f"Unknown Champion with ID {assistId}") for assistId in assistingParticipantIds]

                turret_data = {
                    'teamId': teamId,
                    'towerType': towerType,
                    'timestamp': currentMinute,
                    'position': position,
                    'laneType': laneType,
                    'killerId': killerId,
                    'killerName': killerName,
                    'assistingParticipantIds': assistingParticipantIds,
                    'assistingChampionNames': assistingChampionNames
                }

                if teamId == 100:
                    blueTeamTurrets.append(turret_data)
                elif teamId == 200:
                    redTeamTurrets.append(turret_data)

    return blueTeamTurrets, redTeamTurrets

# Example usage:
blueTeamTurrets, redTeamTurrets = organizeBuildingKillData(data, participantData)

# Display the extracted data for blue team
print("Blue Team Turrets:")
for turret in blueTeamTurrets:
    currentMinute = turret['timestamp']
    print(f"Tower Type: {turret['towerType']}, Timestamp: {currentMinute}, Lane: {turret['laneType']}")
    print(f"Killer: {turret['killerName']}")
    if turret['assistingParticipantIds']:
        print(f"Assisting Champions: {turret['assistingChampionNames']}")
    print('\n')

# Display the extracted data for red team
print("Red Team Turrets:")
for turret in redTeamTurrets:
    currentMinute = turret['timestamp']
    print(f"Tower Type: {turret['towerType']}, Timestamp: {currentMinute}, Lane: {turret['laneType']}")
    print(f"Killer: {turret['killerName']}")
    if turret['assistingParticipantIds']:
        print(f"Assisting Champions: {turret['assistingChampionNames']}")
    print('\n')


Blue Team Turrets:
Tower Type: OUTER_TURRET, Timestamp: 20.04, Lane: BOT_LANE
Killer: Ezreal
Assisting Champions: ['Nautilus']


Tower Type: OUTER_TURRET, Timestamp: 20.91, Lane: MID_LANE
Killer: Sylas


Tower Type: OUTER_TURRET, Timestamp: 29.4, Lane: TOP_LANE
Killer: Unknown Champion with ID 0


Tower Type: INNER_TURRET, Timestamp: 29.9, Lane: TOP_LANE
Killer: Unknown Champion with ID 0
Assisting Champions: ['Shen']


Red Team Turrets:
Tower Type: OUTER_TURRET, Timestamp: 17.34, Lane: MID_LANE
Killer: Veigar


Tower Type: OUTER_TURRET, Timestamp: 20.36, Lane: TOP_LANE
Killer: Jax


Tower Type: INNER_TURRET, Timestamp: 21.02, Lane: TOP_LANE
Killer: Jax


Tower Type: OUTER_TURRET, Timestamp: 24.95, Lane: BOT_LANE
Killer: MasterYi


Tower Type: INNER_TURRET, Timestamp: 28.63, Lane: MID_LANE
Killer: Varus


Tower Type: BASE_TURRET, Timestamp: 31.5, Lane: TOP_LANE
Killer: Jax


Tower Type: Unknown Tower Type, Timestamp: 33.22, Lane: TOP_LANE
Killer: Leona
Assisting Champions: ['Varus']




In [72]:
#blue turrets

bto=True #BLUE TOP OUTER
bti=True #BLUE TOP INNER
bmo=False #BLUE MID OUTER
bmi=False #BLUE MID INNER
bbo=True #BLUE BOT OUTER
bbi=True #BLUE BOT INNER

In [73]:
#red turrets
rto=True #RED TOP OUTER
rti=True #RED TOP INNER
rmo=True #RED MID OUTER
rmi=True #RED MID INNER
rbo=True #RED BOT OUTER
rbi=True #RED BOT INNER

In [74]:
def resetTurrets():
    global bto, bti, bmo, bmi, bbo, bbi, rto, rti, rmo, rmi, rbo, rbi
    bto=True #BLUE TOP OUTER
    bti=True #BLUE TOP INNER
    bmo=True #BLUE MID OUTER
    bmi=True #BLUE MID INNER
    bbo=True #BLUE BOT OUTER
    bbi=True #BLUE BOT INNER
    rto=True #RED TOP OUTER
    rti=True #RED TOP INNER
    rmo=True #RED MID OUTER
    rmi=True #RED MID INNER
    rbo=True #RED BOT OUTER
    rbi=True #RED BOT INNER

In [75]:
def blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi):
    # Initialize caseNumber to 0
    caseNumber = 0
    
    # Check for each case and set the caseNumber accordingly
    if bto and bti and bmo and bmi and bbo and bbi:
        caseNumber = 1
    elif not bto and not bti and not bmo and not bmi and not bbo and not bbi:
        caseNumber = 2
    elif not bmo and not bmi:
        caseNumber = 3
    elif not bmo:
        caseNumber = 4
    elif not bto:
        caseNumber = 5
    elif not bto and not bti:
        caseNumber = 6
    elif not bbo:
        caseNumber = 7
    elif not bbo and not bbi:
        caseNumber = 8
    elif not bmo and not bbo:
        caseNumber = 9
    elif not bto and not bmo:
        caseNumber = 10
    elif not bto and not bbo:
        caseNumber = 11
    elif not bto and not bmo and not bbo:
        caseNumber = 12
    elif not bbo and not bmo and not bmi:
        caseNumber = 13
    elif not bto and not bmo and not bmi:
        caseNumber = 14
    elif not bmo and not bbo and not bbi:
        caseNumber = 15
    elif not bto and not bbo and not bbi:
        caseNumber = 16
    elif not bto and not bti and not bmo:
        caseNumber = 17
    elif not bto and not bti and not bbo:
        caseNumber = 18
    elif bti and bbi:
        caseNumber = 19
    elif bmi and bbi:
        caseNumber = 20
    elif bti and bmi:
        caseNumber = 21
    elif bbo and bbi:
        caseNumber = 22
    elif bto and bti:
        caseNumber = 23
    elif bmo and bmi:
        caseNumber = 24
    elif bbi:
        caseNumber = 25
    elif bti:
        caseNumber = 26
    elif bmi:
        caseNumber = 27
    else:
        caseNumber=404
    
    return caseNumber

# Example usage:
bto = False
bti = False
bmo = False
bmi = False
bbo = False
bbi = False

case_number = blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi)
print("Case number:", case_number)
 

Case number: 2


In [76]:
def redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi):
    # Initialize caseNumber to 0
    caseNumber = 0
    
    # Check for each case and set the caseNumber accordingly
    if rto and rti and rmo and rmi and rbo and rbi:
        caseNumber = 1
    elif not rto and not rti and not rmo and not rmi and not rbo and not rbi:
        caseNumber = 2
    elif not rmo and not rmi:
        caseNumber = 3
    elif not rmo:
        caseNumber = 4
    elif not rto:
        caseNumber = 5
    elif not rto and not rti:
        caseNumber = 6
    elif not rbo:
        caseNumber = 7
    elif not rbo and not rbi:
        caseNumber = 8
    elif not rmo and not rbo:
        caseNumber = 9
    elif not rto and not rmo:
        caseNumber = 10
    elif not rto and not rbo:
        caseNumber = 11
    elif not rto and not rmo and not rbo:
        caseNumber = 12
    elif not rbo and not rmo and not rmi:
        caseNumber = 13
    elif not rto and not rmo and not rmi:
        caseNumber = 14
    elif not rmo and not rbo and not rbi:
        caseNumber = 15
    elif not rto and not rbo and not rbi:
        caseNumber = 16
    elif not rto and not rti and not rmo:
        caseNumber = 17
    elif not rto and not rti and not rbo:
        caseNumber = 18
    elif rti and rbi:
        caseNumber = 19
    elif rmi and rbi:
        caseNumber = 20
    elif rti and rmi:
        caseNumber = 21
    elif rbo and rbi:
        caseNumber = 22
    elif rto and rti:
        caseNumber = 23
    elif rmo and rmi:
        caseNumber = 24
    elif rbi:
        caseNumber = 25
    elif rti:
        caseNumber = 26
    elif rmi:
        caseNumber = 27
    
    return caseNumber

# Example usage:
rto = True
rti = False
rmo = True
rmi = False
rbo = True
rbi = False

case_number = redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi)
print("Case number:", case_number)


Case number: 0


In [82]:
def getTowerDestroyedData(data, participantData):
    towerData = []

    # Initialize boolean variables for blue team
    bto = bti = bmo = bmi = bbo = bbi = True

    # Initialize boolean variables for red team
    rto = rti = rmo = rmi = rbo = rbi = True

    for frame in data['info']['frames']:
        for event in frame.get('events', []):
            if event['type'] == 'BUILDING_KILL':
                teamId = event['teamId']
                towerType = event.get('towerType', None)
                laneType = event['laneType']

                # Update boolean variables based on tower destruction events
                if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
                    if teamId == 100:
                        if towerType == 'OUTER_TURRET':
                            if laneType == 'TOP_LANE':
                                bto = False  # blue top outer destroyed
                            elif laneType == 'MID_LANE':
                                bmo = False  # blue mid outer destroyed
                            elif laneType == 'BOT_LANE':
                                bbo = False  # blue bot outer destroyed
                        elif towerType == 'INNER_TURRET':
                            if laneType == 'TOP_LANE':
                                bti = False  # blue top inner destroyed
                            elif laneType == 'MID_LANE':
                                bmi = False  # blue mid inner destroyed
                            elif laneType == 'BOT_LANE':
                                bbi = False  # blue bot inner destroyed
                        # Call blueDefCaseCalc with updated boolean variables
                        blueDefCaseCalc(bto, bti, bmo, bmi, bbo, bbi)
                    elif teamId == 200:
                        if towerType == 'OUTER_TURRET':
                            if laneType == 'TOP_LANE':
                                rto = False  # red top outer destroyed
                            elif laneType == 'MID_LANE':
                                rmo = False  # red mid outer destroyed
                            elif laneType == 'BOT_LANE':
                                rbo = False  # red bot outer destroyed
                        elif towerType == 'INNER_TURRET':
                            if laneType == 'TOP_LANE':
                                rti = False  # red top inner destroyed
                            elif laneType == 'MID_LANE':
                                rmi = False  # red mid inner destroyed
                            elif laneType == 'BOT_LANE':
                                rbi = False  # red bot inner destroyed
                        # Call redDefCaseCalc with updated boolean variables
                        redDefCaseCalc(rto, rti, rmo, rmi, rbo, rbi)

                # Append tower data to the list
                towerData.append({
                    'teamId': teamId,
                    'towerType': towerType,
                    'laneType': laneType
                })

    return towerData

# Example usage:
towerData = getTowerDestroyedData(data, participantData)

# Display the extracted data
for tower in towerData:
    teamId = tower['teamId']
    towerType = tower['towerType']
    laneType = tower['laneType']

    if towerType in ['OUTER_TURRET', 'INNER_TURRET']:
        print(f"Team: {teamId}, Tower Type: {towerType}, Lane: {laneType}")


Team: 200, Tower Type: OUTER_TURRET, Lane: MID_LANE
Team: 100, Tower Type: OUTER_TURRET, Lane: BOT_LANE
Team: 200, Tower Type: OUTER_TURRET, Lane: TOP_LANE
Team: 100, Tower Type: OUTER_TURRET, Lane: MID_LANE
Team: 200, Tower Type: INNER_TURRET, Lane: TOP_LANE
Team: 200, Tower Type: OUTER_TURRET, Lane: BOT_LANE
Team: 200, Tower Type: INNER_TURRET, Lane: MID_LANE
Team: 100, Tower Type: OUTER_TURRET, Lane: TOP_LANE
Team: 100, Tower Type: INNER_TURRET, Lane: TOP_LANE
Team: 200, Tower Type: INNER_TURRET, Lane: BOT_LANE
